# Test for Utils

In [1]:
from PyNetwork import utils
import pyopencl as cl
import pyopencl.array as cl_array
import numpy as np

n = 10
m = 100
d = 20

platform = cl.get_platforms()
devices = platform[0].get_devices()
context = cl.Context(devices)
queue = cl.CommandQueue(context)

In [2]:
# Set random data
X = np.random.rand(20, 10).astype(np.float32)
Y = np.random.rand(10, 100).astype(np.float32)

# Send data to the gpu
X_gpu = cl_array.to_device(queue, X)
Y_gpu = cl_array.to_device(queue, Y)

gpu_maths = utils.ArrayFunctions(context, queue)
gpu_matmul = utils.NaiveMatMul(context, queue)
gpu_transpose = utils.GPUTranspose(context, queue)

# Test the result of matrix multiplication
matmul_result = gpu_matmul.naiveMatmul(X_gpu, Y_gpu)
np.testing.assert_almost_equal(matmul_result.get(), X @ Y, decimal=5)

In [3]:
b = np.random.rand(10).astype(np.float32)

# Send data to the gpu
b_gpu = cl_array.to_device(queue, b)

# Test the result of vector addition
pred = gpu_maths.addVector(X_gpu, b_gpu)
np.testing.assert_almost_equal(pred.get(), X + b, decimal=5)

# Test the result of vector subtraction
pred = gpu_maths.addVector(X_gpu, -b_gpu)
np.testing.assert_almost_equal(pred.get(), X - b, decimal=5)

# Test the result of vector multiplication
pred = gpu_maths.mulVector(X_gpu, b_gpu)
np.testing.assert_almost_equal(pred.get(), X * b, decimal=5)

# Test the result of vector division
pred = gpu_maths.divVector(X_gpu, b_gpu)
np.testing.assert_almost_equal(pred.get(), X / b, decimal=5)

In [4]:
# Set random data
prev_z = np.random.rand(d, m).astype(np.float32)
delta = np.random.rand(d, n).astype(np.float32)

# Send data to the gpu
prev_z_gpu = cl_array.to_device(queue, prev_z)
delta_gpu = cl_array.to_device(queue, delta)
delta_gpu_T = cl_array.transpose(delta_gpu)

# Test the result of gradients
weight_result = gpu_matmul.naiveMatmul(delta_gpu_T, prev_z_gpu)
np.testing.assert_almost_equal(weight_result.get(), np.ascontiguousarray(delta.T) @ prev_z, decimal=5)
bias_result = gpu_maths.rowSumUp(delta_gpu)
np.testing.assert_almost_equal(bias_result.get(), np.sum(delta, axis=0), decimal=5)

In [5]:
# Set random data
W = np.random.rand(n, m).astype(np.float32)
x = np.random.rand(d, m).astype(np.float32)
b = np.random.rand(n).astype(np.float32)
input_width, _ = x.shape 

# Send data to the gpu
W_gpu = cl_array.to_device(queue, W)
W_gpu_T = cl_array.transpose(W_gpu)
x_gpu = cl_array.to_device(queue, x)
b_matrix = np.tile(b, (input_width, 1))
b_gpu = cl_array.to_device(queue, b_matrix)

pred = gpu_matmul.naiveMatmul(x_gpu, W_gpu_T) + b_gpu
np.testing.assert_almost_equal(pred.get(), x @ W.T + b, decimal=5)

In [6]:
# Set random data
x = -1 * np.random.rand(100, 10).astype(np.float32)
x_gpu = cl_array.to_device(queue, x)

# Test the result of the function sign()
(gpu_maths.clarray_sign(x_gpu).get() == np.sign(x)).all()

True

In [7]:
# Test the effect of contiguous
delta = np.random.rand(d, m).astype(np.float32)
delta_gpu = cl_array.to_device(queue, delta)
delta_gpu_F = cl_array.to_device(queue, np.asfortranarray(delta))
delta_gpu_T = cl_array.transpose(delta_gpu)
weight_grad = gpu_matmul.naiveMatmul(delta_gpu_T, prev_z_gpu)
np.testing.assert_almost_equal(weight_grad.get(), np.ascontiguousarray(delta.T) @ prev_z, decimal=5)

In [8]:
# Test the result of rowMean
bias_mean = gpu_maths.rowMean(delta_gpu)
np.testing.assert_almost_equal(bias_mean.get(), np.mean(delta, axis=0), decimal=5)

# Test the result of rowStd
bias_std = gpu_maths.rowStd(delta_gpu)
np.testing.assert_almost_equal(bias_std.get(), np.std(delta, axis=0), decimal=5)

In [9]:
# Test the result of rowArgmax
x = np.random.rand(100, 10, 10).astype(np.float32)
x_gpu = cl_array.to_device(queue, x)

arg_max = gpu_maths.rowArgmax(x_gpu)
np.testing.assert_almost_equal(arg_max.get(), np.argmax(x, axis=-1))

In [10]:
x = np.random.rand(100, 20).astype(np.float32)
x_gpu = cl_array.to_device(queue, x)
x_transpose = gpu_transpose.transpose(x_gpu)
np.testing.assert_almost_equal(x_transpose.get(), x.T)
x_transpose.flags.c_contiguous

True